In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
# chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [3]:
url='https://www.accessmba.com/events'

try:
    driver.get(url)
except:
    pass
time.sleep(5)

soup=bs(driver.page_source,'lxml')
items=soup.find_all('article',{'class':'event top-event'})
len(items)

cards=[]
for m in items:
    da=' '.join(m.find('div',{'class':'event-info'}).find('p').text.split())
    lo=' '.join(m.find('div',{'class':'event-info'}).find('a').text.split())
    link='https://www.accessmba.com'+m.find('a')['href']
    cards.append([lo,link,da])


In [76]:

access=[]
for k in cards:
    lo=k[0]
    link=k[1]
    da=k[2]
    try:
        driver.get(link)
        soupn=bs(driver.page_source,'lxml')
    except:
        pass
    title=' '.join(soupn.find('div',{'class':'info'}).find('span',{'class':re.compile('event-type hidden-lg.+')}).text.split())
    datiloc=' '.join(soupn.find('div',{'class':'info'}).find('h1').text.split()).split('|')
    loc,dat=[a.strip() for a in datiloc]
    dat=dat.replace('–','-').replace('|','').replace('to','-').replace('TO','-').replace('c-ber','ctober').replace('C-BER','CTOBER').strip()
    for f in ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','Mon','Tue','Wed','Thu','Fri','Sat','Sun']:
        if f in dat:
            dat=dat.replace(f,'').strip()
    
    if dat=='':
        start_date=end_date=''
    else:
        mad=dat.strip()
    if '-' in mad:
        st=mad.split('-')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('-')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif 'and'in mad:
        st=mad.split('and')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()
        en=mad.split('and')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()

    elif '&' in mad:
        st=mad.split('&')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()
        en=mad.split('&')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()

    else:
        st=mad.strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()
        en=mad.strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()
   
    if any(c.isalpha() for c in st)==False:
    #      print('leg')
        pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en
    #                        02        April      2022
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
     #   print('awujale')
        start=st
        end=en

    #*                        April       02      2022
    elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
    #       print('ala')
        pa=re.search('(\d{4})',st).group()
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()
        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba     

    #                      02     2022
    elif re.search('\d{1,2}\s+\d{4}',en):
    #      print('kum')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',st).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    #                      02         April
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
    #        print('is')
        pa=re.search('(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en

    #*                      April           02
    elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
    #      print('bad')
        if not re.search('(\d{4})',en):
            en=en+' '+str(datetime.today().year)
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    else:
    #     print('shik')
        start=end=''
    if start=='':
        start_date=end_date=''
    else:
        pick=[start, end]
        try:
            spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
        except:
            spl_dt_obj = [datetime.strptime(v, '%d %b %Y') for v in pick]
        date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
        start_date=date_[0]
        end_date=date_[1]
    
    ag=soupn.find('div',{'class':'schedule-times-content'}).find_all('div',{'class':'schedule-times-item'})[:-1]
    
    agen=[]
    for p in ag:
        if ' '.join(p.find('div',{'class':'schedule-times-item-time'}).text.split())=='':
            pass
        else:
            if len(p.find('div',{'class':'schedule-times-item-time'}).text.split('-'))==2:

                at=' '.join(p.find('div',{'class':'schedule-times-item-time'}).text.split()).replace('–','-').split('-')[0].strip().replace(' ','')
                if any(c.isalpha() for c in at)==True:
                    rt=at
                else:
                    rt=datetime.strptime(at, '%H:%M').strftime('%I:%M%p')

                bt=' '.join(p.find('div',{'class':'schedule-times-item-time'}).text.split()).replace('–','-').split('-')[1].split('(')[0].strip().replace(' ','')
                if any(c.isalpha() for c in bt)==True:
                    qt=bt
                else:
                    qt=datetime.strptime(at, '%H:%M').strftime('%I:%M%p')

            elif len(p.find('div',{'class':'schedule-times-item-time'}).text.split('-'))==1:

                at=' '.join(p.find('div',{'class':'schedule-times-item-time'}).text.split()).replace('–','-').split('-')[0]
                if any(c.isalpha() for c in at)==True:
                    rt=at
                else:
                    rt=datetime.strptime(at, '%H:%M').strftime('%I:%M%p')

                qt=''

            titl=' '.join(p.find('div',{'class':'schedule-times-item-label'}).text.split())

            arrai={
                'start_time':rt,
                'end_time':qt,
                'day':start_date,
                'desc':'',
                'title':titl

            }
            agen.append(arrai)
    if agen==[] or agen=='':
        agendalist=''
    else:
        agendalist=json.dumps(agen,ensure_ascii=False)
        
        
    if ag=='' or ag==[]:
        time_=''
    else:
        xx=' '.join(ag[0].find('div',{'class':'schedule-times-item-time'}).text.split()).replace('–','-').split('(')[0].strip().split('-')[0].strip().replace(' ','')
        if any(c.isalpha() for c in xx)==True:
            stt=xx
        else:
            stt=datetime.strptime(xx, '%H:%M').strftime('%I:%M%p')
            
        yy=' '.join(ag[-1].find('div',{'class':'schedule-times-item-time'}).text.split()).replace('–','-').split('(')[0].strip().split('-')[-1].strip().replace(' ','')
        if ' '.join(ag[-1].find('div',{'class':'schedule-times-item-time'}).text.split())=='':
            yy=' '.join(ag[-2].find('div',{'class':'schedule-times-item-time'}).text.split()).replace('–','-').split('(')[0].strip().split('-')[-1].strip().replace(' ','')
        if ' '.join(ag[-2].find('div',{'class':'schedule-times-item-time'}).text.split())=='':
            yy=' '.join(ag[-3].find('div',{'class':'schedule-times-item-time'}).text.split()).replace('–','-').split('(')[0].strip().split('-')[-1].strip().replace(' ','')
        if ' '.join(ag[-3].find('div',{'class':'schedule-times-item-time'}).text.split())=='':
            yy=xx
        if any(c.isalpha() for c in yy)==True:
            ett=yy
        else:
            ett=datetime.strptime(yy, '%H:%M').strftime('%I:%M%p')
            
            
        z=''
        time_st={
            'type':'general',
            'Start_time':stt,
            'end_time':ett,
            'timezone':z,
            'days':'all'
        }
        time_=json.dumps([time_st],ensure_ascii=False)
    ven=' '.join(soupn.find('div',{'class':'venue-address'}).text.split())
    if 'We recommend using a desktop computer' in ven or 'stable Internet connection' in ven or 'online' in ven:
        ven=''
        
    def gu(luc):
        google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
        time.sleep(randint(0,3))
        driver.get(google_url_for_location)
        time.sleep(4)
        try:
            google_map_url=driver.find_element_by_id('lu_map').click()
        except:
            try:
                google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
            except:
                google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
        time.sleep(1)
        google_map_url=driver.current_url
#                 print(google_map_url)
        return(google_map_url)
        ######################################
    def get_google_map_url(location):
        try:
            return(gu(location))
        except:
            try:
                return(gu(location))
            except:
                sha=location.split(',')
                try:
                    gu(sha[-3])
                except:
                    try:
                        gu(sha[-2])
                    except:
                        try:
                            gu(sha[-1])
                        except Exception as e:
                            print(location, "; url didn't go through")
                            print(e)
                            return("")


    def countr(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    try:
                        google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
                    except:
                        google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                        time.sleep(randint(0,3))
                        driver.get(google_url_for_location)
                        time.sleep(4)
                        google_map_url=driver.find_element_by_id('lu_map').click()
            time.sleep(1)
            try:
                country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
            except:
                country=driver.find_element_by_class_name('bwoZTb').text
#                 print(country)
            return(country)
        except Exception as e:
            print(locale, ";country didn't go through")
            print(e)
            return("")

    org_name='Access MBA'
    org_web='https://www.accessmba.com/'
    org_pro='Access MBA educates, matches and connects prospective MBA applicants for individual meetings with admissions directors of accredited and top-ranked business schools around the world. Join the highly personalised Access MBA One-to-One events online or onsite with strict Covid-19 safety measures ensured.'
    logo=''    
    sponsor=''
    Speakerlist=''
    type_=''
    category=''
    event_info=''
    ticket_list=''
    event_web=link
    mail_=json.dumps(['projectmanagers@accessmba.com'])
    if 'nline' in loc or 'ff Campus' in loc or loc=='' or loc==' ' or loc=='-' or loc=='Virtual' or 'nspecified' in loc or 'irtual' in loc or 'ebinar' in loc:
        on_off='1'
        city=''
        venue=''
        country=''
        googlePlaceUrl=''
    else:
        
        on_off='0'
        city=loc.split(',')[0].strip()
        country=loc.split(',')[-1].strip()
        if ven=='':
            venue=loc
        else:
            venue=ven
        googlePlaceUrl=get_google_map_url(venue)
        
        
        
        
    access.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsor,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])
    

In [77]:
len(access)

60

In [78]:
access

[['https://www.accessmba.com/events/city/nairobi',
  'Online One-to-One MBA Event',
  '2022-10-26',
  '2022-10-26',
  '[{"type": "general", "Start_time": "5:00PM", "end_time": "8:30PM", "timezone": "", "days": "all"}]',
  '',
  '',
  'Access MBA educates, matches and connects prospective MBA applicants for individual meetings with admissions directors of accredited and top-ranked business schools around the world. Join the highly personalised Access MBA One-to-One events online or onsite with strict Covid-19 safety measures ensured.',
  'Access MBA',
  'https://www.accessmba.com/',
  '',
  '',
  '[{"start_time": "5:00PM", "end_time": "5:30PM", "day": "2022-10-26", "desc": "", "title": "Alumni panel discussion"}, {"start_time": "5:00PM", "end_time": "5:30PM", "day": "2022-10-26", "desc": "", "title": "Online One-to-One meetings* with business schools"}]',
  '',
  '',
  'USA ONLINE (West coast)',
  'USA ONLINE (West coast)',
  'USA ONLINE (West coast)',
  'https://www.accessmba.com/event

In [79]:
access_dict=access


access_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=access_dict)
access_df.to_csv("access.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [ ]:
# driver.switch_to.window(driver.window_handles[0])

In [ ]:
# s=1
# for k in cards:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[0])
#     s+=1
# print(s+1)

In [ ]:
# for k in range(1,31):
#     time.sleep(1)
#     driver.switch_to.window(driver.window_handles[k])
